In [1]:
pricing_hostname = 'https://pricing.api.santecapital.dev'

In [2]:
from sc_pricing import ScPricing

pricing_client = ScPricing(hostname='https://pricing.api.santecapital.com')

In [3]:
import sc_trading_cal
import datetime as dt

In [4]:
from sc_pricing import ScPricing
import os
import datetime as dt
import pandas as pd


class PricingProvider:

    def __init__(self, pricing_client, base_url):
        self.pricing_client = pricing_client
        self.base_url = base_url

    def init_client(self):
        # Access the environment variable for the api key
        # Initialize client
        self.client = self.pricing_client(hostname=self.base_url,
                                          as_of=dt.datetime.now(
                                              dt.timezone.utc),
                                          with_cache=False)

    async def get_price_history(self, trading_item_ids, date_range):
        """
        Format trading_item_ids and return pricing history for a given set of dates
        """
        self.init_client()
        keys = pd.Series(trading_item_ids).apply(lambda x: int(x)).to_list()
        prices = await self.client.get_prices(
            trade_date_range=date_range,
            trading_item_ids=keys,
        )
        await self.client.http_client.close()
        return prices.reset_index()

In [5]:
pricing_provider = PricingProvider(
        pricing_client=ScPricing,
        base_url='https://pricing.api.santecapital.com',
    )

In [6]:
from typing import List

In [7]:
async def get_daily_data(trading_item_ids: List, date_range):
    prices = await pricing_provider.get_price_history(
            trading_item_ids = trading_item_ids, 
            date_range = date_range)
    return prices

In [ ]:
df = await get_daily_data([6179710], sc_trading_cal.get_trade_day_range(start=dt.date(2011, 5, 2), end=dt.date(2022, 2, 18)) )



In [9]:
df.to_csv('SPX-Price-Daily.csv')

In [9]:
df

,trading_item_id,pricing_date,open,high,low,mid,close,bid,ask,volume,adjustment_factor,split_adjustment_factor,dividend_adjustment,vwap
0,6179710,2011-05-02,111.335451,111.424799,110.425729,110.649098,110.645037,110.645037,110.653159,126310033.0,None,1.0,0.812253,110.864345
1,6179710,2011-05-03,110.433851,110.620669,109.690640,110.242972,110.247033,110.238910,110.247033,138445808.0,None,1.0,0.812253,110.133318
2,6179710,2011-05-04,110.198298,110.247033,109.028654,109.520067,109.516006,109.516006,109.524128,182821163.0,None,1.0,0.812253,109.637844
3,6179710,2011-05-05,108.906816,109.613476,108.045829,108.545364,108.525058,108.541303,108.549425,227065699.0,None,1.0,0.812253,109.028654
4,6179710,2011-05-06,109.605353,110.165808,108.208279,108.983980,109.004287,108.979919,108.988042,222951264.0,None,1.0,0.812253,109.645966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2716,6179710,2022-02-14,439.920000,441.600000,435.340000,439.055000,439.020000,439.050000,439.060000,121675400.0,None,1.0,1.000000,438.900000
2717,6179710,2022-02-15,443.730000,446.280000,443.180000,446.075000,446.100000,446.070000,446.080000,88395750.0,None,1.0,1.000000,444.670000
2718,6179710,2022-02-16,443.930000,448.055000,441.940000,446.625000,446.600000,446.620000,446.630000,84399800.0,None,1.0,1.000000,444.960000
2719,6179710,2022-02-17,443.220000,446.565200,436.420000,437.115000,437.060000,437.110000,437.120000,101870300.0,None,1.0,1.000000,439.980000


In [36]:
async def algoseek(year, month, day, tickers):
    '''
    year: int 
    month: int
    tickers : list [str], tickers
    
    '''
    import datetime as dt
    import sc_trading_cal
    from sc_pricing import ScPricing
    pricing_service_host = 'https://pricing.api.santecapital.com'
    pricing_client = ScPricing(hostname= pricing_service_host)

    prices_df = await pricing_client.get_prices(
        source="algoseek",
        trade_date_range=sc_trading_cal.get_trade_day_range(start=dt.date(2021, 11, 13), end=dt.date(2022, 2, 18)),
        resolution='minute',
        adjusted=True,
        adjustment_types=["DIV", "SD"],
        entity_id_type = 'ticker',
        entity_ids= tickers,
    )
    return prices_df

In [40]:
df2 = await algoseek(2021,11,12, ['SPY'])

In [39]:
df.to_csv("SPX_minute_2011_05_02-2021-11-12.csv")

In [41]:
df2

ticker trade_date              open_bar_time  \
ticker trade_date                                                
SPY    2011-05-02    SPY 2011-05-02  2011-05-02T13:30:00+00:00   
       2011-05-02    SPY 2011-05-02  2011-05-02T13:31:00+00:00   
       2011-05-02    SPY 2011-05-02  2011-05-02T13:32:00+00:00   
       2011-05-02    SPY 2011-05-02  2011-05-02T13:33:00+00:00   
       2011-05-02    SPY 2011-05-02  2011-05-02T13:34:00+00:00   
...                  ...        ...                        ...   
       2021-11-12    SPY 2021-11-12  2021-11-12T20:55:00+00:00   
       2021-11-12    SPY 2021-11-12  2021-11-12T20:56:00+00:00   
       2021-11-12    SPY 2021-11-12  2021-11-12T20:57:00+00:00   
       2021-11-12    SPY 2021-11-12  2021-11-12T20:58:00+00:00   
       2021-11-12    SPY 2021-11-12  2021-11-12T20:59:00+00:00   

                   open_bid_price  open_bid_size  open_ask_price  \
ticker trade_date                                                  
SPY    2011-05-02        111.3266        89500.0        111.3347   
       2011-05-02        111.2535         7800.0        111.2616   
       2011-05-02        111.2779        64600.0        111.2860   
       2011-05-02        111.2941        56900.0        111.3023   
       2011-05-02        111.2779        14600.0        111.2860   
...                           ...            ...             ...   
       2021-11-12        465.4912         8100.0        465.5011   
       2021-11-12        465.4513        10200.0        465.4613   
       2021-11-12        465.5111        20800.0        465.5211   
       2021-11-12        465.5609        32700.0        465.5709   
       2021-11-12        465.6307         6200.0        465.6407   

                   open_ask_size                  first_trade_time  \
ticker trade_date                                                    
SPY    2011-05-02         3600.0  2011-05-02T13:30:00.030000+00:00   
       2011-05-02       133200.0  2011-05-02T13:31:00.024000+00:00   
       2011-05-02        24700.0  2011-05-02T13:32:00.041000+00:00   
       2011-05-02        17300.0  2011-05-02T13:33:00.091000+00:00   
       2011-05-02        62000.0  2011-05-02T13:34:00.240000+00:00   
...                          ...                               ...   
       2021-11-12          900.0  2021-11-12T20:55:00.006000+00:00   
       2021-11-12         5900.0  2021-11-12T20:56:00.019000+00:00   
       2021-11-12          700.0  2021-11-12T20:57:00.007000+00:00   
       2021-11-12          900.0  2021-11-12T20:58:00.004000+00:00   
       2021-11-12        19800.0  2021-11-12T20:59:00.003000+00:00   

                   first_trade_price  first_trade_size  ... downtick_volume  \
ticker trade_date                                       ...                   
SPY    2011-05-02           111.3347             200.0  ...        151435.0   
       2011-05-02           111.2616             100.0  ...         55039.0   
       2011-05-02           111.2819             200.0  ...         60555.0   
       2011-05-02           111.3023             100.0  ...         25502.0   
       2011-05-02           111.2779             100.0  ...         27544.0   
...                              ...               ...  ...             ...   
       2021-11-12           465.5011              34.0  ...        316165.0   
       2021-11-12           465.4613              57.0  ...        549596.0   
       2021-11-12           465.5211               7.0  ...        281404.0   
       2021-11-12           465.5709             100.0  ...        821808.0   
       2021-11-12           465.6307             100.0  ...        454796.0   

                   repeat_uptick_volume  repeat_downtick_volume  \
ticker trade_date                                                 
SPY    2011-05-02              705600.0                793798.0   
       2011-05-02              343081.0                283638.0   
       2011-05-02              273533.0                331752.0   
